# Back to refactoring titan algorithms

- Work on loading and syncing running algos
- Work on signals and distributed human intervention
- Work on buy and start for reacting to signals
- Work on exchange based harvester signals
- Work on backtesting
- Normalize candles that are realtime based, i.e. round to the next nearest fraction of an hour
- Storing results with more data points like catalyst pickles
- Need to add drawdown and slippage to stats and profits

# Library Notes

- https://github.com/kylejusticemagnuson/pyti/tree/master/pyti
- https://github.com/mrjbq7/ta-lib
- https://cryptotrader.org/talib
- https://github.com/mrjbq7/ta-lib/issues/17

# Found

- dual moving xmr-btc bittex 5m 3-15

# Bugs

- 1m candles throw list index error
- capital_base gets ignored if less than purchase cost
- default candles from places like bitfinex start at the first point in history therefore need a start, end range
- Candle already contained in DB, retrying...

In [ ]:
from os import getcwd
from kryptobot.portfolio.manager import Manager

config = getcwd() + '/config.json'
manager = Manager(config=config)

params = {
#     'strategy': 'dema_crossover',
    'strategy': 'dual_simple_moving_average',
    'type': 't2',
    'default': {
        'interval': '5m',
        'exchange': 'cryptopia',
        'base_currency': 'PCN',
        'quote_currency': 'BTC',
        'is_simulated': True,
#         'start': '2017-06-15',
#         'end': '2018-06-15',
    },
    'limits': {
        'capital_base': 1000,
        'order_quantity': 100,
        'position_limit': 1000,
        'profit_target_percentage': 1.2,
        'fixed_stoploss_percentage': .95,
        'trailing_stoploss_percentage': .90
    },
    'custom': {
        'short_window': 3,
        'long_window': 24,
#         'threshold': 0.02
    },
    'portfolio': {
        'name': 'default'
    }
}

manager.run_strategy(params)

In [7]:
from os import getcwd
from kryptobot.portfolio.manager import Manager

config = getcwd() + '/config.json'
manager = Manager(config=config)

params = {
    'strategy': 'kitchen_sink_analysis',
    'type': 't2',
    'default': {
        'interval': '5m',
        'exchange': 'bittrex',
        'base_currency': 'XMR',
        'quote_currency': 'BTC',
        'is_simulated': True,
    },
    'limits': {
        'capital_base': 1000,
        'order_quantity': 100,
        'position_limit': 1000,
        'profit_target_percentage': 1.2,
        'fixed_stoploss_percentage': .95,
        'trailing_stoploss_percentage': .90
    },
    'custom': {
        'period': 26
    },
    'portfolio': {
        'name': 'default'
    }
}

manager.run_strategy(params)

In [2]:
from os import getcwd
from kryptobot.portfolio.manager import Manager

config = getcwd() + '/config.json'
manager = Manager(config=config)

results = manager.get_results('17ae1473-3b66-4d6a-9601-0648bcfe0df3')
# results.head(200)
results.tail(200)

,ad,aroon_down,aroon_up,atr,atrp,base_balance,close,exit_balance,high,low,open,positions,quote_balance,rsi,timestamp,volume
618,-15.951705,3.846154,3.846154,0.000725,3.350311,0,0.021634,1000.0,0.021874,0.021634,0.021696,0,1000,76.162747,2018-06-28 02:30:00,15.951705
619,-1.089122,7.692308,7.692308,0.000700,3.240765,0,0.021600,1000.0,0.021634,0.021600,0.021634,0,1000,72.912584,2018-06-28 02:35:00,1.089122
620,-2.075518,3.846154,3.846154,0.000700,3.233232,0,0.021650,1000.0,0.021824,0.021615,0.021615,0,1000,74.877908,2018-06-28 02:40:00,3.137431
621,-6.260904,3.846154,3.846154,0.000616,2.848952,0,0.021622,1000.0,0.021650,0.021616,0.021650,0,1000,70.882759,2018-06-28 02:45:00,10.002248
622,31.434801,3.846154,3.846154,0.000644,2.946577,0,0.021846,1000.0,0.021846,0.021816,0.021816,0,1000,75.647462,2018-06-28 02:50:00,31.434801
623,9.688736,11.538462,11.538462,0.000644,2.952710,0,0.021800,1000.0,0.021800,0.021648,0.021648,0,1000,72.988059,2018-06-28 03:05:00,9.688736
624,-5.837779,7.692308,7.692308,0.000644,2.982543,0,0.021582,1000.0,0.021750,0.021582,0.021750,0,1000,62.508396,2018-06-28 03:10:00,5.837779
625,7.974653,3.846154,3.846154,0.000644,2.966313,0,0.021700,1000.0,0.021700,0.021561,0.021582,0,1000,65.213168,2018-06-28 03:15:00,7.974653
626,-6.868117,11.538462,11.538462,0.000643,2.984014,0,0.021553,1000.0,0.021700,0.021553,0.021700,0,1000,59.821459,2018-06-28 03:20:00,6.868117
627,3.503402,7.692308,7.692308,0.000643,2.964801,0,0.021693,1000.0,0.021700,0.021553,0.021700,0,1000,62.739663,2018-06-28 03:25:00,3.889025


In [ ]:
import talib
help(talib.RSI)